In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(celldex)
    library(ShinyCell)
    library(dplyr)
    library(SingleR)
    library(stringr)
    library(tidyr)
    library(tibble)
})

In [84]:
obj1 <- readRDS("data/PBMC.single.cell.RDS")
counts <- GetAssayData(obj1)
metadata <- obj1@meta.data
obj1 <- CreateSeuratObject(counts = counts, meta.data = metadata)

# Clean columns before merge
obj1@meta.data$Tissue <- 'PBMC'
obj1@meta.data$CDR3_NT <- obj1@meta.data$cdr3s_nt
obj1@meta.data$Donor <- obj1@meta.data$Sample
obj1@meta.data$THERAPY <- obj1@meta.data$Therapy
obj1$Type <- gsub("HC", "Healthy Control", obj1$Type)
obj1$Type <- gsub("CC_NI", "CC Non-Inflamed", obj1$Type)
obj1$Type <- gsub("CC_I", "CC Inflamed", obj1$Type)

obj1@meta.data <- obj1@meta.data[, !(colnames(obj1@meta.data) %in%  c('cdr3s_nt' ,'Sample', 'Therapy', 'Pool'))]

dim(obj1)
head(obj1@meta.data)

[1] 33538 27084

,orig.ident,nCount_RNA,nFeature_RNA,Type,Inflamed,NANCY,UCIES,Cluster,Tissue,CDR3_NT,Donor,THERAPY
,<fct>,<dbl>,<int>,<chr>,<chr>,<dbl>,<dbl>,<fct>,<chr>,<fct>,<chr>,<chr>
AAACCTGAGAGGTAGA-1_1,SeuratProject,1996.110,1115,CC Non-Inflamed,No,0,0,CD4+ T-Cell,PBMC,TRA:TGTGCAGCAAGTACCCCGGGCTCAACCCTGGGGAGGCTATACTTT;TRB:TGTGCCAGCAGCTTAGGCTATGCACAGAGCTTAACTGAAGCTTTCTTT,PRISE1,Dual
AAACCTGAGATCCCGC-1_1,SeuratProject,2640.887,1672,CC Non-Inflamed,No,0,0,NK-Cell,PBMC,TRB:TGCGCCAGCACTCCCCGCTGGACGGGAGCCCAAAACAATGAGCAGTTCTTC,PRISE1,Dual
AAACCTGAGCCACGCT-1_1,SeuratProject,1780.494,843,Ambigious,Ambigious,0,0,CD4+ T-Cell,PBMC,TRB:TGTGCCAGCACCTACAGGGGGAGCGGAAACACCATATATTTT,Ambigious,No Treatment
AAACCTGAGCGATCCC-1_1,SeuratProject,2004.645,1133,UC,Yes,2,3,CD4+ T-Cell,PBMC,NA,IBD3947,No Treatment
AAACCTGAGGATATAC-1_1,SeuratProject,2371.318,1271,CC Inflamed,Yes,3,5,NK-Cell,PBMC,NA,GI4059,Dual
AAACCTGAGGCATGGT-1_1,SeuratProject,1489.082,646,CC Inflamed,Yes,3,5,CD4+ T-Cell,PBMC,NA,GI4059,Dual


In [85]:
obj2 <- readRDS("data/CD3.single.cell.RDS")
counts <- GetAssayData(obj2)
metadata <- obj2@meta.data
obj2 <- CreateSeuratObject(counts = counts, meta.data = metadata)

# Clean columns before merge
obj2@meta.data$Tissue <- 'Colon'
obj2@meta.data <- obj2@meta.data[, !(colnames(obj2@meta.data) %in%  c('Dataset', 'Reaction'))]

dim(obj2)
head(obj2@meta.data)

[1] 14790 72561

,orig.ident,nCount_RNA,nFeature_RNA,Type,Donor,CDR3_NT,STEROIDS,AGE,SEX,THERAPY,UCEIS,LAG,SYMPTOMS,Phase,Cluster,Tissue
,<fct>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<fct>,<chr>
AAACCTGAGAGCTGGT-1_1,SeuratProject,1452.835,569,CPI Control,GI4373,TRB:TGCAGTGCTAACAGCGGGAGCCCGCCAGATACGCAGTATTTT;TRA:TGCCTCGTGGGTGACATACAAAACCAGGGAGGAAAGCTTATCTTC,No,63,M,Monotherapy,0,28,8,G2M,Th1,Colon
AAACCTGAGCACAGGT-1_1,SeuratProject,1380.564,510,CPI Colitis,PRISE50,NA,No,70,M,Dual Therapy,4,16,4,G1,Th1,Colon
AAACCTGAGGGTCGAT-1_1,SeuratProject,1625.697,627,CPI Colitis,PRISE50,TRB:TGCGCCAGCAGCTTCTGGCGGGACAGGGACACAGATACGCAGTATTTT,No,70,M,Dual Therapy,4,16,4,G2M,Tfh,Colon
AAACCTGAGTCAAGCG-1_1,SeuratProject,1740.345,738,UC Inflamed,GI5696,TRB:TGTGCCAGCAGCTTTGGCCCCCAAAGACAGGGGGATGGCTACACCTTC;TRA:TGTGCTGTGGGTGCGTGGGATGCCAGACTCATGTTT,No,69,M,None,2,NA,240,S,Tregs,Colon
AAACCTGCAAGGGTCA-1_1,SeuratProject,1379.969,514,CPI Control,PRISE22,TRB:TGTGCCAGCAGCTTAGGGCAGAGTAATGAAAAACTGTTTTTT;TRA:TGTGCAGCAAGTAACTATGGAAAGCTGACATTT,Yes,69,M,Dual Therapy,0,6,3,G1,Th1,Colon
AAACCTGCAATCAGAA-1_1,SeuratProject,1535.496,585,CPI Control,PRISE22,NA,Yes,69,M,Dual Therapy,0,6,3,G2M,Th1,Colon


In [90]:
obj <- merge(x = obj1, y = obj2)
obj <- JoinLayers(obj)
dim(obj)
obj
save(obj, file=file.path("objects", "00.rda"))

Warning message:
“Some cell names are duplicated across objects provided. Renaming to enforce unique cell names.”


[1] 33538 99645

An object of class Seurat 
33538 features across 99645 samples within 1 assay 
Active assay: RNA (33538 features, 0 variable features)
 1 layer present: counts

In [91]:
# Update metadata
obj@meta.data$Stim <- obj@meta.data$Type
obj@meta.data$Cell_Type <- obj@meta.data$Cluster

columns_to_keep <-  c('Donor', 'Tissue', 'Stim', 'STEROIDS', 'AGE', 'SEX', 'THERAPY', 'Phase', 'Cell_Type')
obj@meta.data <- obj@meta.data[, columns_to_keep, drop = FALSE]
head(obj@meta.data)
save(obj, file=file.path("objects", "01.rda"))

,Donor,Tissue,Stim,STEROIDS,AGE,SEX,THERAPY,Phase,Cell_Type
,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
AAACCTGAGAGGTAGA-1_1_1,PRISE1,PBMC,CC Non-Inflamed,NA,NA,NA,Dual,NA,CD4+ T-Cell
AAACCTGAGATCCCGC-1_1_1,PRISE1,PBMC,CC Non-Inflamed,NA,NA,NA,Dual,NA,NK-Cell
AAACCTGAGCCACGCT-1_1_1,Ambigious,PBMC,Ambigious,NA,NA,NA,No Treatment,NA,CD4+ T-Cell
AAACCTGAGCGATCCC-1_1_1,IBD3947,PBMC,UC,NA,NA,NA,No Treatment,NA,CD4+ T-Cell
AAACCTGAGGATATAC-1_1_1,GI4059,PBMC,CC Inflamed,NA,NA,NA,Dual,NA,NK-Cell
AAACCTGAGGCATGGT-1_1_1,GI4059,PBMC,CC Inflamed,NA,NA,NA,Dual,NA,CD4+ T-Cell


In [ ]:
load(file = "objects/01.rda")
options(future.globals.maxSize = 4000 * 1024^2)
obj <- SCTransform(obj, verbose = TRUE)
save(obj, file=file.path("objects", "02.rda"))

In [ ]:
# Load annotation
monaco.ref <- celldex::MonacoImmuneData()
blueprint.ref <- celldex::BlueprintEncodeData()

# Blueprint annnotation
sce <- LayerData(obj)
blueprint.main <- SingleR(test = sce, assay.type.test = 1, ref = blueprint.ref, labels = blueprint.ref$label.main)
obj@meta.data$blueprint.main <- blueprint.main$pruned.labels
save(obj, file=file.path("objects", "04.rda"))

# Monaco annontation
sce <- LayerData(obj)
monaco.fine <- SingleR(test = sce, assay.type.test = 1, ref = monaco.ref, labels = monaco.ref$label.fine)
obj@meta.data$monaco.fine <- monaco.fine$pruned.labels
save(obj, file=file.path('objects', '05.rda'))


In [95]:
?createConfig

createConfig             package:ShinyCell             R Documentation

_C_r_e_a_t_e _a _s_h_i_n_y_c_e_l_l _c_o_n_f_i_g _d_a_t_a._t_a_b_l_e

_D_e_s_c_r_i_p_t_i_o_n:

     Create a shinycell config data.table containing (i) the
     single-cell metadata to display on the Shiny app, (ii) ordering of
     factors / categories of categorical metadata and (iii) colour
     palettes associated with each metadata.

_U_s_a_g_e:

     createConfig(obj, meta.to.include = NA, legendCols = 4, maxLevels = 50)
     
_A_r_g_u_m_e_n_t_s:

     obj: input single-cell object for Seurat (v3+) /
          SingleCellExperiment data or input file path for h5ad / loom
          files

meta.to.include: columns to include from the single-cell metadata.
          Default is ‘NA’, which is to use all columns. Users can
          specify the columns to include, which must match one of the
          following:

            • Seurat objects: column names in ‘s

In [92]:
# load(file=file.path('objects', '05.rda'))

# # Colon
# seu <- obj
# seu <- SetIdent(seu, value = "Tissue")
# seu <- subset(seu, Tissue  ==  "Colon")

# # Recluster
# seu <- RunPCA(seu, npcs = 30, verbose = TRUE)
# seu <- RunUMAP(seu, reduction = "pca", dims = 1:20)
# seu <- FindNeighbors(seu, reduction = "pca", dims = 1:20)
# seu <- FindClusters(seu, resolution = 0.5)

scConf1 = createConfig(seu, maxLevels = 60)
makeShinyFiles(seu, scConf1,
             gene.mapping = TRUE,
             shiny.prefix = "sc1",
             shiny.dir = "colon_pbmc_uc_scRNAseq_GSE189040/",
             gex.assay = "SCT")

# # PBMC
# seu <- obj
# seu <- SetIdent(seu, value = "Tissue")
# seu <- subset(seu, Tissue  ==  "PBMC")

# # Recluster
# seu <- RunPCA(seu, npcs = 30, verbose = TRUE)
# seu <- RunUMAP(seu, reduction = "pca", dims = 1:20)
# seu <- FindNeighbors(seu, reduction = "pca", dims = 1:20)
# seu <- FindClusters(seu, resolution = 0.5)

# scConf2 = createConfig(seu)
# makeShinyFiles(seu, scConf2,
#              gene.mapping = TRUE,
#              shiny.prefix = "sc2",
#              shiny.dir = "colon_pbmc_uc_scRNAseq_GSE189040/",
#              gex.assay = "SCT")

PC_ 1 
Positive:  IGKC, IGHA1, RPL7, IGLC2, RPL27A, RPL13A, RPS17, RPS20, MT-ND2, MT-ND4 
	   RPS10, RPL37A, MT-CO3, RPL23, JCHAIN, RPL21, MT-CYB, RPS11, IGLC3, RPS2 
	   RPS16, RPL31, MT-ND1, MT-ATP6, TRAC, RPSA, NBEAL1, RPL10A, RPL4, SLC25A6 
Negative:  JUND, UBE2S, FOS, DUSP1, ZFP36, NR4A1, MTRNR2L12, IFITM1, JUN, MIF 
	   HSPA1A, JUNB, MTRNR2L8, HSPA1B, DNAJA1, CRIP1, TNF, DNAJB1, TAF10, GADD45B 
	   HSPA8, SERTAD1, LINC00910, FOSB, HOOK2, EGR1, SIK3, TUBB4B, IER5L, TMEM238 
PC_ 2 
Positive:  MT-ATP8, RACK1, MT-ND4L, ATP5F1E, ATP5MC2, S100A8, ATP5MG, CST3, SELL, S100A9 
	   LYZ, NOP53, ELOB, PTPRCAP, TYROBP, FYB1, FXYD5, TRIR, TXNIP, ATP5F1D 
	   AES, AIF1, CCR7, C1orf162, FCN1, RARRES3, FCER1G, LEF1, SEPT7, VSIR 
Negative:  CCL5, CCL4, CCL4L2, IGHA1, IGKC, IFNG, CD69, XCL2, XCL1, JCHAIN 
	   IGLC2, KLF6, RGS1, ZFP36, GZMA, NKG7, JUND, CD8B, IGLC3, SPRY1 
	   CD8A, CCL3, DUSP1, TNFSF9, CD3D, TRGC2, HCST, HOPX, FOS, GZMK 
PC_ 3 
Positive:  NKG7, CCL4, GZMA, GZMB, CCL5, CCL4L2, IFNG,

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 72561
Number of edges: 2252424

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9253
Number of communities: 18
Elapsed time: 17 seconds


ID,UI,fID,fCL,fRow,grp,dimred
<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
Tissue,Tissue,Colon,black,1,FALSE,FALSE
Stim,Stim,CPI Colitis|CPI Control|Healthy Control|UC Inflamed|UC Non-Inflamed,#A6CEE3|#52AF43|#F06C45|#B294C7|#B15928,2,TRUE,FALSE
STEROIDS,STEROIDS,No|Yes,#A6CEE3|#B15928,1,TRUE,FALSE
AGE,AGE,NA,NA,NA,FALSE,FALSE
SEX,SEX,F|M,#A6CEE3|#B15928,1,TRUE,FALSE
THERAPY,THERAPY,Dual Therapy|Monotherapy|None,#A6CEE3|#F06C45|#B15928,1,TRUE,FALSE
Phase,Phase,G1|G2M|S,#A6CEE3|#F06C45|#B15928,1,TRUE,FALSE
Cell_Type,Cell_Type,CD103- HAVCR2+|CD103+ HAVCR2+|CD103+ TRM|CD103+ ZNF683+|Cycling|FGFBP2+ Effector|GZMH+ Effector|GZMK+ Effector|IFN Response|MAITs|Naive CD4+|Naive CD8+|Tfh|Th1|Th17 PD1-|Th17 PD1+|Tph|Tregs|γδ-rich 1|γδ-rich 2,#A6CEE3|#579CC7|#3688AD|#8BC395|#89CB6C|#40A635|#919D5F|#F99392|#EB494A|#E83C2D|#F79C5D|#FDA746|#FE8205|#E39970|#BFA5CF|#8861AC|#917099|#E7E099|#DEB969|#B15928,5,TRUE,FALSE
nCount_SCT,nCount_SCT,NA,NA,NA,FALSE,FALSE


PC_ 1 
Positive:  S100A8, CST3, LYZ, S100A9, TYROBP, FCN1, AIF1, S100A12, FCER1G, SERPINA1 
	   CSTA, SPI1, ATP5F1E, VCAN, CD14, LST1, CD68, MNDA, MS4A6A, ATP5MC2 
	   CFD, RACK1, HLA-DRA, ELOB, ATP5MG, CTSS, VSIR, CSF3R, ATP2B1-AS1, TMEM176B 
Negative:  CD3D, IL32, CD69, CD3E, JUNB, JUND, RGS1, DUSP2, EVL, HSPA8 
	   UBB, TNFAIP3, CCL5, BTG1, RPS29, IGKC, DUSP1, TSC22D3, ZFP36, DNAJB1 
	   TRAC, RGCC, CXCR4, KLRB1, TRBC2, HSP90AA1, KLF6, NR4A2, IGHA1, JUN 
PC_ 2 
Positive:  RACK1, AES, GNLY, NOP53, MT-ND4L, FGFBP2, SEPT1, MT-ATP8, RARRES3, TXNIP 
	   ATP5F1E, PTPRCAP, ATP5MG, SEPT7, SEPT6, ATP5MC2, AC243960.1, TRIR, RIPOR2, CTSW 
	   MT-CYB, SPON2, KIAA1551, IFITM1, PRF1, FYB1, FCGR3A, KLRF1, RPSA, SELL 
Negative:  S100A8, LYZ, S100A9, DUSP1, CST3, S100A12, FCN1, JUND, FOS, KLF6 
	   SERPINA1, CSTA, CD14, AIF1, ZFP36, FOSB, MS4A6A, RGS1, LST1, MNDA 
	   VCAN, CFD, SRGN, SPI1, SAT1, CD68, NR4A1, RGS2, IGKC, NR4A2 
PC_ 3 
Positive:  GNLY, FGFBP2, FCGR3A, PRF1, SPON2, NKG7, GZMB, KLRF1, 

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 27084
Number of edges: 961622

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9181
Number of communities: 21
Elapsed time: 4 seconds


ID,UI,fID,fCL,fRow,grp,dimred
<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
Donor,Donor,2935_CC|3052_CC|3128_CC|4069_CC|4448_CC|4680_CC|4738_CC|Ambigious|GI4059|GI4158|GI4381|IBD3947|PRISE1|PRISE11|PRISE13|PRISE4|PRISE5|TIP200|TIP278_H,#A6CEE3|#5399C6|#3F8EAA|#99CD91|#79C360|#3E9F32|#B89B74|#F47676|#E52829|#F06C45|#FDB762|#FE901E|#ED8F47|#CCAFCA|#9471B4|#825D99|#DDD399|#E0BE6D|#B15928,5,TRUE,FALSE
Tissue,Tissue,PBMC,black,1,FALSE,FALSE
Stim,Stim,Ambigious|CC Inflamed|CC Non-Inflamed|Healthy Control|UC,#A6CEE3|#52AF43|#F06C45|#B294C7|#B15928,2,TRUE,FALSE
STEROIDS,STEROIDS,NA,NA,NA,FALSE,FALSE
AGE,AGE,NA,NA,NA,FALSE,FALSE
SEX,SEX,NA,NA,NA,FALSE,FALSE
THERAPY,THERAPY,Dual|Mono|No Treatment,#A6CEE3|#F06C45|#B15928,1,TRUE,FALSE
Phase,Phase,NA,NA,NA,FALSE,FALSE
Cell_Type,Cell_Type,B-cells|CD4+ T-Cell|CD8+ T-Cell|Cell Cycle|Classical Monocytes (CD14+)|DCs|Erythroid|Mast|Megakaryocytes|NK-Cell|Non-Classical Monocytes (CD16+)|pDCs|Plasma,#A6CEE3|#2A7FB7|#99CD91|#52AF43|#B89B74|#ED4F50|#F06C45|#FDA440|#ED8F47|#B294C7|#825D99|#F8F18F|#B15928,4,TRUE,FALSE


In [93]:
makeShinyCodesMulti(
  shiny.title = "Ulcerative and Immune Checkpoint Therapy Colitis scRNAseq", shiny.footnotes = NULL,
  shiny.prefix = c("sc1", "sc2"),
  shiny.headers = c("Colon", "PBMC"), 
  shiny.dir = "colon_pbmc_uc_scRNAseq_GSE189040/") 

In [94]:
system("R -e \"shiny::runApp('colon_pbmc_uc_scRNAseq_GSE189040')\"")